In [ ]:
# !pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("joeddav/distilbert-base-uncased-go-emotions-student")
model = AutoModelForSequenceClassification.from_pretrained("joeddav/distilbert-base-uncased-go-emotions-student")

# Define the emotions (GoEmotions has 27 labels, ignoring the extra class)
emotions = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity",
    "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear",
    "gratitude", "grief", "joy", "love", "nervousness", "optimism", "pride", "realization",
    "relief", "remorse", "sadness", "surprise"
]

text = """Jesus continued:ÊÒThere was a man who had two sons.ÊThe younger one said to his father, ÔFather, give me my share of the estate.ÕÊSo he divided his propertyÊbetween them.
ÒNot long after that, the younger son got together all he had, set off for a distant country and there squandered his wealthÊin wild living.ÊAfter he had spent everything, there was a severe famine in that whole country, and he began to be in need.ÊSo he went and hired himself out to a citizen of that country, who sent him to his fields to feed pigs.ÊHe longed to fill his stomach with the pods that the pigs were eating, but no one gave him anything.ÊÒWhen he came to his senses, he said, ÔHow many of my fatherÕs hired servants have food to spare, and here I am starving to death! ÊI will set out and go back to my father and say to him: Father, I have sinnedÊagainst heaven and against you.ÊI am no longer worthy to be called your son; make me like one of your hired servants.ÕÊSo he got up and went to his father.
ÒBut while he was still a long way off, his father saw him and was filled with compassion for him; he ran to his son, threw his arms around him and kissed him.
 ÒThe son said to him, ÔFather, I have sinned against heaven and against you.ÊI am no longer worthy to be called your son.Õ
 ÒBut the father said to his servants, ÔQuick! Bring the best robeÊand put it on him. Put a ring on his fingerÊand sandals on his feet.Ê Bring the fattened calf and kill it. LetÕs have a feast and celebrate.Ê For this son of mine was dead and is alive again;Êhe was lost and is found.Õ So they began to celebrate.
 ÒMeanwhile, the older son was in the field. When he came near the house, he heard music and dancing.Ê So he called one of the servants and asked him what was going on.Ê ÔYour brother has come,Õ he replied, Ôand your father has killed the fattened calf because he has him back safe and sound.Õ
 ÒThe older brother became angryÊand refused to go in. So his father went out and pleaded with him.Ê But he answered his father, ÔLook! All these years IÕve been slaving for you and never disobeyed your orders. Yet you never gave me even a young goat so I could celebrate with my friends. But when this son of yours who has squandered your propertyÊwith prostitutesÊcomes home, you kill the fattened calf for him!Õ
 ÒÔMy son,Õ the father said, Ôyou are always with me, and everything I have is yours.Ê32ÊBut we had to celebrate and be glad, because this brother of yours was dead and is alive again; he was lost and is found.ÕÓ"""

sentences = [s.strip() for s in text.split('.') if s.strip()]
print(sentences)

for sentence in sentences:
    print(f'Working on sentence: {sentence}')
    
    # Tokenize the input text
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform emotion detection
    outputs = model(**inputs)

    # Get the logits
    logits = outputs.logits

    # Print the shape of the logits
    print(f"Logits shape: {logits.shape}")

    # Apply sigmoid to get probabilities
    probabilities = torch.sigmoid(logits).detach().numpy()[0]

    # Ignore the extra logit
    probabilities = probabilities[:27]  # Slice to match the 27 emotions

    # Debugging: Print lengths to ensure consistency
    print(f"Length of emotions: {len(emotions)}")
    print(f"Length of probabilities: {len(probabilities)}")

    # Get the predicted emotions with probabilities for all emotions
    predicted_emotions = [(emotions[i], probabilities[i]) for i in range(len(probabilities))]

    # Print the results
    print(f"Text: {sentence}")
    print("Predicted Emotions and Probabilities:")
    for emotion, prob in predicted_emotions:
        print(f"  {emotion}: {prob:.4f}")
